# Simulation command generation from simulation sheet
>'It's turtles all the way down!'

Plan: 
- Have array of sample settings, maybe optimized to give comparable $\sigma t$ to give comparable signal strength.

In [ ]:
from definitions import *
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt

In [ ]:
# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
Rs = [2e-6, 300e-9, 50e-9]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]

# Source sample (thickness) pairing (L0, DL, R, t)
source_sample = [(4.321, 0.04321, 2e-6, 1e-3), (8, 0.8, 2e-6, 1e-3), (4.321, 0.04321, 300e-9, 10e-3), (8, 0.8, 300e-9, 5e-3), (4.321, 0.04321, 50e-9, 10e-3), (8, 0.8,50e-9, 10e-3)]

thickness_map = {
    (4.321, 2e-6):  1e-3,
    (8, 2e-6):  1e-3,
    (4.321, 300e-9): 10e-3,
    (8, 300e-9): 5e-3,
    (4.321, 50e-9): 10e-3,
    (8, 50e-9): 10e-3,
}
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
print(f"Constant parameters: drho = {delta_rho * 1e-14}e14 m^-2; phi = {phi}")
for (L0, DL, R, t) in source_sample:
    # print(thickness_map[(L0, R)])
    print(F"R = {R * 1e9}nm, t = {round(t * 1e3,2)}mm, lambda = {L0} Å:")
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    permissible = st < 0.8 and st >= 0.1
    print(f"\ts*t: {round(st, 4)}\n\t0.1 <= s*t <= 0.8: {permissible}")
    sigma = s_t(R,t, L0 * 1e-10) / t
    t_optimum = 0.38 / sigma

In [ ]:
from util import *
from instrument import Instrument

instrs = load_instruments('simulations_new.csv')

N_foil = 10000000
N_iso_wsp = 100000000

N_steps = 30

shift_Ls = True

with open('simulate.sh', 'w') as f:
    f.write('#!/bin/bash\n# Simulation script automatically generated by simulation-driver.ipynb, use this to create variants of it\n')
    f.write('rm -rf data\n')
    for instr in instrs[3:-6]:
        prec_type = instr.prec_type
        if instr.prec_type == 'wsp':
            prec_type = 'iwsp'

        # if instr.prec_type == 'foil':
        #     continue
        # By = 0.01
        
        d_min, d_max = instr.delta_range()
        if shift_Ls:
            L_s_scaling = 20e-9 / d_min
            L_s_new = instr.L_s * L_s_scaling

            # print(L_s_new)
            theta_a_max = 15e-3
            L_min = round(detector_size / (2 * np.tan(theta_a_max)),4)
            L_s_new_corrected = max(L_s_new, L_min)
            print(L_s_new_corrected, L_min, L_s_new)
            instr.L_s = L_s_new_corrected
        d_min, d_max = instr.delta_range()
        print("-------------------------\n"+str(instr))
        for R in Rs:
            t = thickness_map[(instr.L0 * 1e10, R)]
            mode = 'GPU'
            if instr.prec_type == 'foil':
                mode = 'CPU'
            # print(f"delta_y range: {round(d_min * 1e9,2)} - {round(d_max * 1e9, 2)}nm")
            # print(f"\tRange of interest for R = {R * 1e9}nm: 0 - {3 * R * 1e9}")
            print(f"\tR = {R * 1e9}nm:")
            print(f"\t\tRange of interest:  {round(0.1 * R * 1e9)} - {round(3 * R * 1e9)}nm")

            

            overlap = find_overlap((d_min, d_max), (0.0, 3 * R))
            if overlap == None:
                print("No overlap!")
                f.write(f'echo "Skipping {instr.id}_{int(R * 1e10)} due to non-overlapping ranges!\n')

                raise Exception("No overlap!")
            else:
                (a,b) = overlap
                fraction = (b - a) / (3 * R) * 100
                log_fraction = log_overlap_percentage((d_min, d_max), (0.1 * R, 3 * R))
                print(f"\t\tOverlap: {round(a * 1e9,2)} - {round(b * 1e9, 2)}nm ({round(fraction,1)}% of linear range, {round(log_fraction,1)}% of log range)")
                d_min_B_field, d_max_B_field = instr.delta_range_B_field()
                max_ratio = b / d_max_B_field
                min_ratio = a / d_min_B_field
                By_min = instr.By_min * min_ratio
                # The simulation parameter By range corresponds to the setting of B1
                # The maximal setting is when B1 = Bmax * L2 / L1 so that
                # B2 = B1 * L1/L2 = Bmax, having the greater field
                # Take the ratio from this depending on the overlap
                By_max = instr.By_max * instr.L_2 / instr.L_1 * max_ratio
                print(f"\t\t=>Simulating B field range {By_min} - {By_max}")
                # print(By_max)
                if instr.prec_type == 'foil':
                    N = N_foil
                else:
                    N = N_iso_wsp
                sim_descr = f"# Name: {instr.name}, R = {round(R * 1e9)} nm\n# Simulating delta range {round(a * 1e9, 4)} - {round(b * 1e9, 4)} nm using B field range {round(By_min * 1e3,4)} - {round(By_max * 1e3,4)} mT (limits: {round(instr.By_min * 1e3,4)} - {round(instr.By_max * 1e3,4)} mT)"

                # print(sim_cmd)
                f.write(f'{sim_descr}\n')
                sim_cmd = f"./full-simulation.sh {N} 4 {N_steps} {By_min},{By_max} {instr.L0 * 1e10} {instr.DL* 1e10} {R * 1e10} {t} {instr.theta_0} {instr.L_s} {prec_type} {prec_type}_empty {mode}; rm -rf data_{instr.id}_{int(R * 1e10)}; mv data data_{instr.id}_{int(R * 1e10)}"
                # print(sim_cmd)
                f.write(f'{sim_cmd}\n')


# Solve the equation
$$R_{pixel} = \frac{\sin(\pi p f)}{\pi p f} = 0.75$$

In [ ]:
from sympy import sin, nsolve, Symbol, pi
x = Symbol('x')

V_reduction = 0.7
solution = float(nsolve(sin(x) / x  - V_reduction, 0.4))
x_range = np.linspace(-4 * np.pi, 4 * np.pi, 10000)
y = np.sin(x_range) / x_range
plt.plot(x_range, y)
plt.axvline(solution, linestyle='--')
plt.axhline(V_reduction, linestyle='--')
plt.grid()
# solution

In [ ]:
f_min = solution / (np.pi * detector_pixel_size)
f_min, f_min * 2e-10 * 1.6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

optimal_min = []
optimal_max = []

num_samples = 1  # Number of samples to average
L_maxes = np.linspace(3.0, 9.0, 12)
for L_max in L_maxes:
    delta_ranges = []
    for _ in range(num_samples):
        pg_instr = optimize_instrument(PG=False, L_max=L_max)
        delta_range = np.array(pg_instr.delta_range())
        delta_ranges.append(delta_range)
    
    # Calculate the average delta_range over all samples
    avg_delta_range = np.max(delta_ranges, axis=0)
    
    optimal_min.append(avg_delta_range[0])
    optimal_max.append(avg_delta_range[1])


In [ ]:
plt.title(r'$\delta_{max}$ as function of instrument length')
# plt.plot(L_maxes, optimal_min, label=r'$\delta_{min}$')
plt.plot(L_maxes, np.array(optimal_max) * 1e9, '.', label='Wollaston prism')
plt.xlabel(r'$L_{1,max}$ [m]')
plt.ylabel(r'$\delta_{max}$ [nm]')
plt.legend()
plt.grid()
# plt.axhline(5e-6, linestyle='--', color='red')
plt.show()

In [ ]:
delta_range = pg_instr.delta_range()
# length_penalty = 0.01 * L_1 / L_1_0

fitness = log_overlap_percentage(delta_range, target_interval) 
np.log(delta_range), np.log(target_interval)
print(fitness)
# a,b = np.log(delta_range)
# a,b

# Milk sample (from dairy paper)

In [ ]:
delta_rho = 2.0e14 # 1/m^2 (?)
phi = 0.036
R = 50e-9
t = 2e-3
for (L0, _) in sources:
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    print(st)

# Q setting
The final puzzlepiece: the realization that you need to simulate higher $Q$ for smaller characteristic lengths! The circle complete, Wim would have told me this tomorrow and thought again of how I told him how I first didn't understand what $Q$ meant in literature. 
$R = 10000$Å is simulated with Qmind = 0.00003,Qmaxd = 0.001 in units Å-1. Then Qmaxd = 0.0005 should strictly be enough for R = 20000 and the others should be larger!
